In [ ]:
!pip install --user slither-analyzer

In [7]:
from parserPackage import parser
from parserPackage.traceTree import TraceTree
from main import storeATrace, readATrace, storeATrace2

txHash = "0x0a1abf7620d6fca54f5d6818960438695ef1ab7732b3a58a57fc7347d87da527"
vmparser = parser.VmtraceParser()
vmparser.printStack()

globalState = vmparser.setupGlobalState(txHash)

status, origin = vmparser.setupGlobalState(txHash)
metaData = {"meta": True, "txHash": txHash, "status": status, "origin": origin}
   
metaTraceTree = TraceTree(metaData) 
metaTraceTree.visualizeASE()


{'meta': True, 'txHash': '0x0a1abf7620d6fca54f5d6818960438695ef1ab7732b3a58a57fc7347d87da527', 'status': 2, 'origin': '0x365e6e422f4b682586ee34b061cb1e38ffda7968'}



'[Meta Info]  TxHash: 0x0a1abf7620d6fca54f5d6818960438695ef1ab7732b3a58a57fc7347d87da527   tx.origin: 0x365e6e422f4b682586ee34b061cb1e38ffda7968\n'

In [ ]:
storeATrace2(txHash)
# temp = readATrace(txHash)
# print(f"{temp}")
# metaTraceTree.decodeABIStorage(temp["structLogs"])
# print(metaTraceTree.visualizeASE_decoded())



## Adding nodes manually to the TraceTree 
Haven't properly tested this though

In [8]:
# Assuming you have parsed a transaction trace and extracted relevant information
root_info = {"type": "transaction", "from": "0x...", "to": "0x...", "value": 100}
root_trace_tree = TraceTree(root_info)

# Example of an internal call
internal_call_info = {"type": "call", "from": "0x...", "to": "0x...", "value": 50}
internal_trace_tree = TraceTree(internal_call_info)

# Add the internal call to the root trace tree
root_trace_tree.addInternalCall(internal_trace_tree, depth=1)

# Now, printing the root_trace_tree should show a hierarchical structure
print(root_trace_tree)

{'type': 'transaction', 'from': '0x...', 'to': '0x...', 'value': 100}
  {'type': 'call', 'from': '0x...', 'to': '0x...', 'value': 50}



## Analyzing transaction and getting a TraceTree from it

In [19]:
# Import necessary modules and functions
from main import storeATrace, readATrace
from parserPackage.parserGlobal import analyzeOneTxGlobal
import os


# Define the transaction hash you want to analyze
txHash = "0xed2e82bb59e2ea39bfdc7d08ae2f7fcad7200e00163f6e3440b9a5d72fc3ef5d"

# Step 1: Fetch and store a transaction trace
storeATrace(txHash)

# Step 2: Read the stored transaction trace
temp = readATrace(txHash)

# Optional: Print the first few logs to verify
for ii in range(0, 10):
    print(temp['structLogs'][ii])

# Step 3: Parse the transaction trace
print("=================== Trace Tree =====================")
SCRIPT_DIR = os.getcwd()  # Gets the current working directory
path = os.path.join(SCRIPT_DIR, "cache", f"{txHash}.json.gz")
metaTraceTree = analyzeOneTxGlobal(txHash, path)
print(metaTraceTree.visualizeASE())

# Step 4: Decode function ABI and Storage Accesses (dynamically track SHA3)
print("=================== Decoded Trace Tree =====================")
metaTraceTree.decodeABIStorage(temp['structLogs'])
print(metaTraceTree.visualizeASE_decoded())

{'pc': 0, 'op': 'PUSH1', 'gas': 71545, 'depth': 1, 'stack': []}
{'pc': 2, 'op': 'PUSH1', 'gas': 71542, 'depth': 1, 'stack': ['0x80']}
{'pc': 4, 'op': 'MSTORE', 'gas': 71539, 'depth': 1, 'stack': ['0x80', '0x40']}
{'pc': 5, 'op': 'PUSH1', 'gas': 71527, 'depth': 1, 'stack': []}
{'pc': 7, 'op': 'CALLDATASIZE', 'gas': 71524, 'depth': 1, 'stack': ['0x4']}
{'pc': 8, 'op': 'LT', 'gas': 71522, 'depth': 1, 'stack': ['0x4', '0x44']}
{'pc': 9, 'op': 'PUSH2', 'gas': 71519, 'depth': 1, 'stack': ['0x0']}
{'pc': 12, 'op': 'JUMPI', 'gas': 71516, 'depth': 1, 'stack': ['0x0', '0x77']}
{'pc': 13, 'op': 'PUSH4', 'gas': 71506, 'depth': 1, 'stack': []}
{'pc': 18, 'op': 'PUSH29', 'gas': 71503, 'depth': 1, 'stack': ['0xffffffff']}
=================== Trace Tree =====================
[Meta Info]  TxHash: 0xed2e82bb59e2ea39bfdc7d08ae2f7fcad7200e00163f6e3440b9a5d72fc3ef5d   tx.origin: 0x710295b5f326c2e47e6dd2e7f6b5b0f7c5ac2f24
    [CALL] { value:0.00e+00 } 0x0000000000085d4780b73119b644ae5ecd22b376.(830E...0000)